This notebook is the last step before completing Phase 2 of the ACT project by uploading with VanderBot

In [ ]:
import pandas as pd

date_problems_frame = pd.read_csv('issues_with_inception_dates.csv', na_filter=False, dtype = str)
date_problems_list = list(date_problems_frame['act'])
creator_mismatches_frame = pd.read_csv('creator_name_mismatches.csv', na_filter=False, dtype = str)
creator_mismatches_list = list(creator_mismatches_frame['act'])
works_frame = pd.read_csv('abstract_artworks_charlotte_edits.csv', na_filter=False, dtype = str)
#works_frame = works_frame.head(100).copy()
works_frame.head()

## Pull out rows with date problems

In [ ]:
date_problems_output_list = []
remaing_works_list = []
for work_index, work_row in works_frame.iterrows():
    #print(work_row['act'])
    
    if work_row['act'] in date_problems_list: # only try to match if it's on the problem list
        for date_index, date_row in date_problems_frame.iterrows():
            if date_row['act'] == work_row['act']:
                date_problems_output_list.append(work_row)
    else:
        remaing_works_list.append(work_row)

remaing_works_frame = pd.DataFrame(remaing_works_list)

date_problems_output_frame = pd.DataFrame(date_problems_output_list)
date_problems_output_frame.to_csv('date_problems.csv')
date_problems_output_frame.head()


## Pull out rows with missing creator values


In [ ]:
missing_creators_output_list = []
remaing_works_list = []
for work_index, work_row in remaing_works_frame.iterrows():
    #print(work_row['act'])
    
    if work_row['creator'] == '':
        missing_creators_output_list.append(work_row)
    else:
        remaing_works_list.append(work_row)

remaing_works_frame = pd.DataFrame(remaing_works_list)

missing_creators_output_frame = pd.DataFrame(missing_creators_output_list)
missing_creators_output_frame.to_csv('missing_creators.csv')
missing_creators_output_frame.head()

## Pull out rows with creator name mismatches 

In [ ]:
creator_mismatch_output_list = []
remaing_works_list = []
for work_index, work_row in remaing_works_frame.iterrows():
    #print(work_row['act'])
    
    if work_row['act'] in creator_mismatches_list: # only try to match if it's on the problem list
        for date_index, date_row in creator_mismatches_frame.iterrows():
            if date_row['act'] == work_row['act']:
                creator_mismatch_output_list.append(work_row)
    else:
        remaing_works_list.append(work_row)

remaing_works_frame = pd.DataFrame(remaing_works_list)

creator_mismatch_output_frame = pd.DataFrame(creator_mismatch_output_list)
creator_mismatch_output_frame.to_csv('creator_mismatch_problems.csv')
creator_mismatch_output_frame.head()


## Write the works that remain after screening

In [ ]:
remaing_works_frame.to_csv('works_to_write.csv')
remaing_works_frame.head()
